In [ ]:
from datagenerator import *
from unet import *
import tensorflow as tf
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam

In [ ]:
print(tf.keras.__version__,tf.__version__)

In [ ]:
def create_data(no_samples=100,dx=256,dy=256,dz=1):    
    #create training data.
    X=np.random.randn(no_samples,dx,dy,dz) #0 mean 1 std_dev
    #generate reference data (label/true data)
    Y=0.8*X+2   #0.8 std_dev 2 mean
    print(X.shape,Y.shape)
    #generate some validation data
    X_val=np.random.randn(no_samples,dx,dy,dz)
    Y_val=0.8*X_val+2
    print(X_val.shape,Y_val.shape)
    return (X,Y,X_val,Y_val)

In [ ]:
X,Y,X_val,Y_val=create_data(no_samples=5000)

In [ ]:
train_gen=DataGenerator(X=X,Y=Y,batch_size=32,shuffle=False)
val_gen=DataGenerator(X=X_val,Y=Y_val,batch_size=32,shuffle=False)

In [ ]:
def build_model_gpu():
    strategy=tf.distribute.MirroredStrategy(['gpu:0','gpu:1'])
    with strategy.scope():
        input_tensor=Input((256,256,1))
        model=unet(input_tensor,maxpool=False)
        model.compile(optimizer=Adam(),loss='mse',metrics=['accuracy'])
        return model

In [ ]:
def build_model():
    input_tensor=Input((256,256,1))
    model=unet(input_tensor,maxpool=False)
    model.compile(optimizer=Adam(),loss='mse',metrics=['accuracy'])
    return model

In [ ]:
model=build_model_gpu()

In [ ]:
import math
#for next_element in
import time
t0=time.time()
model.fit(train_gen,
                    steps_per_epoch=math.floor(len(X)/32),
                    epochs=100,
                    validation_data=val_gen
                   )
t1=time.time()

In [ ]:
print("time_taken: ",t1-t0)